In [2]:
:dep tch = { version = "0.1.7", path = "../../LaurentMazare/tch-rs" }
// :dep plotters = { git = "https://github.com/38/plotters", features = ["evcxr"] }
:dep plotters = { path = "../../38/plotters", features = ["evcxr"] }
:dep qr = { path = "./" }

In [3]:
use tch::{nn, nn::ModuleT, Tensor, Kind};
use qr::{create_model, train_qr_model, load_data};
use plotters::prelude::*;

In [10]:
fn plot_result(xs: &Tensor, ys: &Tensor) -> plotters::evcxr::SVGWrapper {
    evcxr_figure((640, 480), |root| {
    
        // The following code will create a chart context
        let mut chart = ChartBuilder::on(&root)
            .caption("Training data", ("Arial", 20).into_font())
            .x_label_area_size(40)
            .y_label_area_size(40)
            .build_ranged(12f64..22f64, -2f64..3f64)?;
    
        chart.configure_mesh()
            .disable_x_mesh()
            .disable_y_mesh()
            .draw()?;
        
        // Plot prediction
        let n_points = xs.size()[0];
        for j in 0..=6 {
            chart.draw_series(LineSeries::new((0..n_points).map(|i|
                (xs.double_value(&[i, 0]), ys.double_value(&[i, j]))),
                &BLUE
            ));
        }
        
        // Scatter plot
        let (xs, ys) = load_data().unwrap();
        let n_points = xs.size()[0];
        chart.draw_series((0..n_points).map(|i| Circle::new(
            (xs.double_value(&[i]), ys.double_value(&[i])), 3, BLUE.filled()
        )));

        Ok(())
    }).style("width:60%")
}

In [11]:
fn train() -> plotters::evcxr::SVGWrapper {
    // Device
    let device = tch::Device::cuda_if_available();
    let vs = nn::VarStore::new(device);
    
    // Initialize model
    let n_inputs = 1i64;
    let n_quants = 7i64;
    let model = create_model(&vs.root(), n_inputs, n_quants, vec![8]);

    // Training loop
    // the type impl tch::nn::module::ModuleT cannot currently be persisted
    let model = train_qr_model(model, n_quants, device, &vs);
    
    // Prediction
    let xs = Tensor::arange2(10.0, 24.0, 0.2, (Kind::Float, device)).view_(&[-1, 1]);
    let ys = xs.apply_t(&model, false); // train = false in 2nd arg
    println!("{:?}", ys.size());
    
    // Plot prediction and data
    plot_result(&xs, &ys)
}

In [12]:
train()

Quantiles
 0.1250
 0.2500
 0.3750
 0.5000
 0.6250
 0.7500
 0.8750
[ CPUFloatType{7} ]
iter =     0, loss = 0.3218071460723877
iter =  2000, loss = 0.2108207643032074
iter =  4000, loss = 0.20822012424468994
iter =  6000, loss = 0.20785333216190338
iter =  8000, loss = 0.2078661173582077
iter = 10000, loss = 0.20727649331092834
iter = 12000, loss = 0.20629681646823883
iter = 14000, loss = 0.20462894439697266
iter = 16000, loss = 0.20273444056510925
iter = 18000, loss = 0.20055894553661346
[70, 7]


Training data
 
 
 
-1.5
 
 
 
-1.0
 
 
 
-0.5
 
 
 
0.0
 
 
 
0.5
 
 
 
1.0
 
 
 
1.5
 
 
 
2.0
 
 
 
2.5
 
 
 
 
13.0
 
 
 
14.0
 
 
 
15.0
 
 
 
16.0
 
 
 
17.0
 
 
 
18.0
 
 
 
19.0
 
 
 
20.0
 
 
 
21.0
 
 
 
22.0
 
 
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,174 40,176 40,178 40,180 40,183 40,186 40,190 40,194 40,199 40,204 40,211 52,218 64,227 76,236 88,246 100,256 112,267 124,277 136,286 148,293 160,299 172,304 184,306 196,307 208,307 220,305 232,302 244,299 256,296 268,292 280,288 292,285 304,283 316,281 328,280 340,280 352,280 364,282 376,284 388,287 400,291 412,295 424,300 436,306 448,312 460,318 472,325 484,332 496,339 508,346 520,353 532,360 544,367 556,374 568,381 580,387 592,394 604,400 616,406 628,412 640,418 640,423 640,428 640,433 640,438 640,440 640,440 640,440 640,440 640,440 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,83 40,85 40,88 40,92 40,96 40,100 40,105 40,111 40,118 40,127 40,136 52,147 64,160 76,174 88,189 100,204 112,220 124,236 136,250 148,263 160,273 172,281 184,287 196,291 208,292 220,292 232,290 244,288 256,284 268,281 280,277 292,274 304,271 316,268 328,267 340,266 352,266 364,267 376,269 388,271 400,274 412,278 424,283 436,288 448,293 460,299 472,306 484,312 496,319 508,326 520,332 532,339 544,346 556,353 568,360 580,366 592,373 604,379 616,385 628,391 640,396 640,402 640,407 640,412 640,417 640,421 640,426 640,430 640,434 640,438 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,78 40,81 40,83 40,86 40,89 40,93 40,97 40,102 40,108 40,116 40,124 52,134 64,145 76,157 88,171 100,185 112,199 124,213 136,227 148,238 160,248 172,256 184,262 196,267 208,269 220,270 232,269 244,268 256,266 268,264 280,262 292,260 304,259 316,257 328,257 340,257 352,257 364,259 376,261 388,263 400,267 412,270 424,275 436,279 448,284 460,290 472,295 484,301 496,307 508,313 520,318 532,324 544,330 556,336 568,342 580,347 592,353 604,358 616,363 628,368 640,373 640,378 640,382 640,386 640,390 640,394 640,398 640,401 640,404 640,407 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,70 40,72 40,75 40,77 40,80 40,83 40,87 40,92 40,97 40,104 40,112 52,121 64,131 76,142 88,154 100,167 112,180 124,193 136,205 148,216 160,226 172,234 184,240 196,244 208,247 220,248 232,248 244,248 256,247 268,246 280,245 292,244 304,244 316,243 328,244 340,244 352,246 364,248 376,250 388,253 400,257 412,261 424,265 436,270 448,275 460,281 472,286 484,292 496,297 508,303 520,309 532,315 544,320 556,326 568,331 580,337 592,342 604,347 616,352 628,356 640,361 640,365 640,369 640,373 640,377 640,380 640,383 640,387 640,390 640,392 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,52 40,54 40,56 40,59 40,62 40,65 40,69 40,74 40,79 40,86 40,93 52,101 64,111 76,121 88,133 100,145 112,157 124,169 136,181 148,191 160,200 172,208 184,214 196,219 208,222 220,224 232,226 244,226 256,227 268,227 280,227 292,227 304,228 316,228 328,230 340,231 352,233 364,236 376,239 388,243 400,246 412,251 424,255 436,260 448,266 460,271 472,277 484,282 496,288 508,293 520,299 532,305 544,310 556,316 568,321 580,326 592,331 604,336 616,341 628,345 640,350 640,354 640,358 640,361 640,365 640,368 640,372 640,375 640,378 640,380 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 40,27 52,32 64,43 76,55 88,68 100,81 112,95 124,109 136,122 148,135 160,146 172,156 184,164 196,171 208,177 220,182 232,186 244,189 256,191 268,194 280,196 292,198 304,201 316,203 328,206 340,209 352,213 364,217 376,221 388,225 400,230 412,235 424,241 436,246 448,252 460,257 472,263 484,269 496,275 508,281 520,287 532,292 544,298 556,303 568,309 580,314 592,319 604,323 616,328 628,333 640,337 640,341 640,345 640,349 640,352 640,356 640,359 640,362 640,365 640,367 "/>
<polyline fill="none" opacity="1" stroke="#0000FF" stroke-width="1" points="